In [1]:
from __future__ import print_function
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
import matplotlib
#matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime

import subprocess
from numpy import genfromtxt
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objs as go
from scipy.spatial import cKDTree
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import bs4
import itertools
import bottleneck as bn
from scipy.signal import lfilter
from itertools import compress
from numpy import inf
import pandas as pd
DATA = np.load('/home/kumarv/tayal007/MODIS_TILES/h08v05.npy')

In [2]:
def readModisfile(path):
    global mask
    file = SD(path, SDC.READ)
    sds_obj = file.select('Land_Cover_Type_1') 
    mask = np.array(sds_obj.get())
    return mask
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

def cleanNDVI(NDVISTACK):
	print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Infinity value in raw data is ", np.count_nonzero(np.isinf(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isinf(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Greater than 1 is ", np.count_nonzero(NDVISTACK > 1),'which is ' ,percentage(np.count_nonzero(NDVISTACK > 1),NDVISTACK.shape),'%')
	print ("Less than -1 is ", np.count_nonzero(NDVISTACK < -1),'which is ' ,percentage(np.count_nonzero(NDVISTACK < -1),NDVISTACK.shape),'%')
	
	CLEANED_DATA = np.copy(NDVISTACK)
	CLEANED_DATA[NDVISTACK == inf] = np.nan
	CLEANED_DATA[NDVISTACK == -inf] = np.nan
	CLEANED_DATA[NDVISTACK > 1] = np.nan
	CLEANED_DATA[NDVISTACK < -1] = np.nan

	print ("After Cleaning Missing value is ", np.count_nonzero(np.isnan(CLEANED_DATA)),'which is ' ,percentage(np.count_nonzero(np.isnan(CLEANED_DATA)),NDVISTACK.shape),'%')
	return CLEANED_DATA;
data2013 = np.copy(DATA[:,:,586:631])
data2013[data2013 < 0] = 0
data2013 = cleanNDVI(data2013)
#data2013 = data2013.reshape(data2013.shape[0] * data2013.shape[1], data2013.shape[2])
mask = readModisfile('/panfs/roc/groups/6/kumarv/tayal007/MODIS_yearly/MCD12Q1.A2013001.h08v05.051.2014308185137.hdf')
crop_data = data2013[mask==12]

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning:

invalid value encountered in less



Missing value in raw data is  16338907 which is  6.3035906635802466 %
Infinity value in raw data is  160282 which is  0.06183719135802469 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning:

invalid value encountered in greater



Greater than 1 is  1946976 which is  0.7511481481481481 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning:

invalid value encountered in less



Less than -1 is  0 which is  0.0 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:20: RuntimeWarning:

invalid value encountered in greater

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:21: RuntimeWarning:

invalid value encountered in less



After Cleaning Missing value is  18285883 which is  7.054738811728395 %


In [3]:
row_mean = np.nanmean(crop_data, axis=1)
inds = np.where(np.isnan(crop_data))
crop_data[inds] = np.take(row_mean, inds[0])
data2013_r = crop_data.reshape(crop_data.shape[0],crop_data.shape[1],1)
print (data2013_r.shape)

(229337, 45, 1)


In [4]:
trainX = data2013_r.reshape(data2013_r.shape[0],data2013_r.shape[1])
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 8  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
input_dim = 45
# this is our input placeholder
input_img = Input(shape=(input_dim,))
# "encoded" is the encoded representation of the input
encoded = Dense(32, activation='relu')(input_img)
encoded = Dense(16, activation='relu')(encoded)
#encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
#decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

encoder = Model(input_img, encoded)
autoencoder.fit(trainX, trainX,
                epochs=50,
                batch_size=2048,
                shuffle=True)

Using TensorFlow backend.


Epoch 1/50
229337/229337 [==============================] - 18s 78us/step - loss: 0.0347
Epoch 2/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0314
Epoch 3/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0291
Epoch 4/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0273
Epoch 5/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0252
Epoch 6/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0225
Epoch 7/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0204
Epoch 8/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0190
Epoch 9/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0177
Epoch 10/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0159
Epoch 11/50
229337/229337 [==============================] - 1s 3us/step - loss: 0.0135
Epoch 12/50
229337/229337 [============

In [6]:
crop = autoencoder.predict(crop_data,2048,1)
noncrop = autoencoder.predict(data2013[mask!=12],2048,1)
encoded_crop = encoder.predict(crop_data,2048,1)
encoded_noncrop = encoder.predict(data2013[mask!=12],2048,1)

5530663/5530663 [==============================] - 3s 1us/step


In [7]:
np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/CROP_8_Auto",crop)
np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/NON_CROP_8_Auto",noncrop)
np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/ENC_CROP_8_Auto",encoded_crop)
np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/ENC_NON_CROP_8_Auto",encoded_noncrop)

In [8]:
trainX = data2013_r

from keras.layers import *
from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector

timesteps=45
input_dim=1
n_dimensions=16

def get_model(n_dimensions):
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(n_dimensions, return_sequences=False, name="encoder")(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True, name='decoder')(decoded)

    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    return autoencoder, encoder

autoencoder, encoder = get_model(n_dimensions)
#autoencoder.compile(optimizer='rmsprop', loss='mse', 
#                    metrics=['acc', 'cosine_proximity'])
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')
history = autoencoder.fit(trainX, trainX, batch_size=65536, epochs=100)


Epoch 1/100
229337/229337 [==============================] - 5s 20us/step - loss: 0.2347
Epoch 2/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.1856
Epoch 3/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.1387
Epoch 4/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0901
Epoch 5/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0485
Epoch 6/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0311
Epoch 7/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0289
Epoch 8/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0287
Epoch 9/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0286
Epoch 10/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0286
Epoch 11/100
229337/229337 [==============================] - 4s 16us/step - loss: 0.0285
Epoch 12/100
229337

In [11]:
encoded_crop = encoder.predict(trainX,2048,1)
encoded_noncrop = encoder.predict(non_crop_data,2048,1)

5530662/5530662 [==============================] - 3s 1us/step


In [12]:
np.save("AUT_CROP_Encoding_8",encoded_crop)
np.save("AUT_NON_CROP_Encoding_8",encoded_noncrop)

In [10]:
non_crop_data = data2013[mask!=12]
row_mean = np.nanmean(non_crop_data, axis=1)
inds = np.where(np.isnan(non_crop_data))
non_crop_data[inds] = np.take(row_mean, inds[0])
#non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]
non_crop_data = non_crop_data.reshape(non_crop_data.shape[0],non_crop_data.shape[1],1)

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning:

Mean of empty slice



In [12]:
crop = autoencoder.predict(trainX,2048,1)
noncrop = autoencoder.predict(non_crop_data,2048,1)
encoded_crop = encoder.predict(trainX,2048,1)
encoded_noncrop = encoder.predict(non_crop_data,2048,1)

5530663/5530663 [==============================] - 65s 12us/step


In [14]:
np.save("/home/kumarv/tayal007/Encodings/LSTM/8/CROP_8_Auto",crop)
np.save("/home/kumarv/tayal007/Encodings/LSTM/8/NON_CROP_8_Auto",noncrop)
np.save("/home/kumarv/tayal007/Encodings/LSTM/8/ENC_CROP_8_Auto",encoded_crop)
np.save("/home/kumarv/tayal007/Encodings/LSTM/8/ENC_NON_CROP_8_Auto",encoded_noncrop)

In [6]:
from keras.utils import plot_model
plot_model(autoencoder, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.